# Avance 2 — Integración con Yelp Fusion API (Miami)

## Proyecto: Conociendo al Cliente 360°
**Autor:** Cristian Fernando García Cadena  
**Fecha:** 23 de Octubre de 2025 

## Objetivo
Conectar a la **Yelp Fusion API**, extraer una muestra representativa de negocios **“restaurants”** para la ciudad seleccionada en el Avance 1 (**Miami**), normalizar los datos a formato tabular y **enriquecer** la base de clientes con **KPI por ciudad** (número de negocios, rating promedio y categoría predominante).  
El foco es **técnico y reproducible**: autenticación, requests, transformación JSON→pandas y merge controlado con el dataset interno.

---

## Contexto (desde el Avance 1)
- Se eligió **Miami** por volumen de clientes y señales favorables (gasto y tasa premium).
- Contamos con la base limpia `data/processed/base_clientes_clean.csv`.
- En este avance incorporamos **información externa** (Yelp) para sumar **contexto local** de oferta gastronómica.

---

## Entradas y salidas

**Entradas**
- API: `https://api.yelp.com/v3/businesses/search` (parámetros: `term=restaurants`, `location=Miami`, `limit=50`).
- Dataset interno: `data/processed/base_clientes_clean.csv` (clientes).

**Salidas**
- **Cache crudo** de la respuesta: `data/external/yelp_raw/miami_p0.json`  
  (evita gastar llamadas durante el desarrollo).
- **Tabla procesada Yelp**: `data/processed/yelp_businesses_miami.csv`.
- **Base enriquecida por persona**: `data/processed/base_clientes_clean_enriched.csv`  
  (agrega: `yelp_n_businesses`, `yelp_avg_rating`, `yelp_top_category`).
- **Notebook de entrega:** `Avance_2_API_Yelp.ipynb`
- **Dataset intermedio enriquecido:** `data/processed/base_clientes_clean_enriched.csv` 

---


In [22]:
import pandas as pd
import requests
import numpy as np
import openpyxl
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# URL de la API
api_url = 'https://api.yelp.com/v3/businesses/search'

# Datos de autenticación de la cuenta de usuario creada previamente
clientid = 'GWOCZh9-BmZxtdsAjr7Gug'
apikey = 'FHVvXoNmTXIl9DuxYis7AV5uLPujm9MLwrhgs5NgvCfaOxd3V6mxt6dQU8eEqYJiGxe816XATx7ufWjbMWqbV-2Uku1jxBJv8BGRC74NroLPl27PDQqs0tDixit-YHYx'

# Cabecera requerida para autenticar la solicitud
headers = {'Authorization': 'Bearer %s' % apikey}

# Ciudad seleccionada (puede venir del dataset del Avance 1)
ciudad = "Miami"

# Consulta de restaurantes en la ciudad
params = {'term': 'restaurants', 'location': ciudad, 'limit': 50}
response = requests.get(api_url, params=params, headers=headers)

# Conversión de la respuesta JSON a diccionario
data = response.json()
data


{'businesses': [{'id': 'K3ukx2e11xTRtYBU01dmrA',
   'alias': 'salty-flame-miami',
   'name': 'Salty Flame',
   'image_url': 'https://s3-media0.fl.yelpcdn.com/bphoto/_6wShTvzfucB5dxFZv4Mpg/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/salty-flame-miami?adjust_creative=GWOCZh9-BmZxtdsAjr7Gug&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=GWOCZh9-BmZxtdsAjr7Gug',
   'review_count': 212,
   'categories': [{'alias': 'asianfusion', 'title': 'Asian Fusion'},
    {'alias': 'steak', 'title': 'Steakhouses'},
    {'alias': 'cocktailbars', 'title': 'Cocktail Bars'}],
   'rating': 4.4,
   'coordinates': {'latitude': 25.76022, 'longitude': -80.19267},
   'transactions': ['pickup', 'delivery'],
   'location': {'address1': '1414 Brickell Ave',
    'address2': None,
    'address3': '',
    'city': 'Miami',
    'zip_code': '33131',
    'country': 'US',
    'state': 'FL',
    'display_address': ['1414 Brickell Ave', 'Miami, FL 33131']},
   'phone': '+1305563897

In [23]:
# === verificación de la respuesta ===
print("Keys principales:", list(data.keys()))
print("Total (reportado por Yelp):", data.get("total"))
print("Negocios devueltos en esta página:", len(data.get("businesses", [])))

# chequeo bare-minimum
assert "businesses" in data, "La respuesta no trae 'businesses'. Revisa headers o la API key."
assert isinstance(data["businesses"], list), "'businesses' no es una lista."


Keys principales: ['businesses', 'total', 'region']
Total (reportado por Yelp): 5600
Negocios devueltos en esta página: 50


A partir de la información obtenida de la API, se procede a generar un df que nos permita almacenar la información sin tener que consultar reiterativamente la API

In [24]:
# === JSON → DataFrame (campos útiles) ===

def yelp_businesses_to_df(payload: dict) -> pd.DataFrame:
    rows = []
    for b in payload.get("businesses", []):
        cats = b.get("categories") or []
        cat_primary = cats[0]["title"] if cats else None

        disp_addr = (b.get("location") or {}).get("display_address") or []
        address = ", ".join(disp_addr) if disp_addr else None

        rows.append({
            "business_id": b.get("id"),
            "name": b.get("name"),
            "rating": b.get("rating"),
            "review_count": b.get("review_count"),
            "price": b.get("price"),  # puede no venir
            "category_primary": cat_primary,
            "lat": (b.get("coordinates") or {}).get("latitude"),
            "lon": (b.get("coordinates") or {}).get("longitude"),
            "address": address,
            "city": (b.get("location") or {}).get("city"),
            "zip_code": (b.get("location") or {}).get("zip_code"),
            "yelp_url": b.get("url"),
        })
    df_biz = pd.DataFrame(rows)

    # limpieza suave y tipos
    if not df_biz.empty:
        df_biz["rating"] = pd.to_numeric(df_biz["rating"], errors="coerce")
        df_biz["review_count"] = pd.to_numeric(df_biz["review_count"], errors="coerce").astype("Int64")
        df_biz["city"] = df_biz["city"].astype("string").str.strip().str.lower()
        df_biz["category_primary"] = df_biz["category_primary"].astype("string")
        df_biz = df_biz.drop_duplicates(subset=["business_id"]).reset_index(drop=True)

    return df_biz

df_yelp = yelp_businesses_to_df(data)
print(df_yelp.shape)
df_yelp.head(3)


(50, 12)


,business_id,name,rating,review_count,price,category_primary,lat,lon,address,city,zip_code,yelp_url
0,K3ukx2e11xTRtYBU01dmrA,Salty Flame,4.4,212,None,Asian Fusion,25.760220,-80.192670,"1414 Brickell Ave, Miami, FL 33131",miami,33131,https://www.yelp.com/biz/salty-flame-miami?adj...
1,oxtMfBGmVNE18pFVuw7lFg,Fratellino,4.8,1827,$$$,Italian,25.749170,-80.260110,"264 Miracle Mile, Coral Gables, FL 33134",coral gables,33134,https://www.yelp.com/biz/fratellino-coral-gabl...
2,UXHxLN3DcDGI57uDIfCuJA,Old's Havana Cuban Bar & Cocina,4.4,3041,$$,Cuban,25.765594,-80.219238,"1442 SW 8th St, Miami, FL 33135",miami,33135,https://www.yelp.com/biz/olds-havana-cuban-bar...


In [25]:
# === Guardar cache crudo ===

# Detectar raíz del proyecto (si estás en notebooks/, sube un nivel)
CWD = Path.cwd()
PROJ_DIR = CWD.parent if CWD.name == "notebooks" else CWD

RAW_DIR = PROJ_DIR / "data" / "external" / "yelp_raw"
RAW_DIR.mkdir(parents=True, exist_ok=True)

cache_path = RAW_DIR / "miami_p0.json"  # 1 sola página (offset 0)
with cache_path.open("w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print("Cache crudo guardado en:", cache_path.resolve())


Cache crudo guardado en: C:\Users\Diego Garcia\Documents\Cristian\DS-SoyHenry\Modulo 1\Código\data\external\yelp_raw\miami_p0.json


In [26]:
# === Exportar CSV procesado ===
PROC_DIR = PROJ_DIR / "data" / "processed"
PROC_DIR.mkdir(parents=True, exist_ok=True)

out_csv = PROC_DIR / "yelp_businesses_miami.csv"
df_yelp.to_csv(out_csv, index=False, encoding="utf-8")

print("CSV procesado guardado en:", out_csv.resolve())
print("Filas guardadas:", len(df_yelp))


CSV procesado guardado en: C:\Users\Diego Garcia\Documents\Cristian\DS-SoyHenry\Modulo 1\Código\data\processed\yelp_businesses_miami.csv
Filas guardadas: 50


Pese a que el ejercicio es limitado, y la API no permite un merge óptimo con la database original, se proponen algunos KPIs para entender el comportamiento de la información para la ciudad escogida: Miami

In [27]:
# === KPIs por ciudad (aquí, Miami) y merge con df base ===

# 1) KPIs simples desde Yelp
kpis_city = (
    df_yelp.groupby("city", dropna=False)
           .agg(
               yelp_n_businesses=("business_id", "count"),
               yelp_avg_rating=("rating", "mean"),
               yelp_top_category=("category_primary", lambda s: s.mode().iloc[0] if len(s.dropna()) else pd.NA)
           )
           .round({"yelp_avg_rating": 2})
)

display(kpis_city)

# 2) Cargar tu DF limpio del Avance 1 (completo, sin filtrar)
base_clean_csv = PROJ_DIR / "data" / "processed" / "base_clientes_clean.csv"
df_base = pd.read_csv(base_clean_csv, encoding="utf-8")

# normalizar ciudad_residencia igual que en Yelp (lower/strip)
df_base["ciudad_residencia"] = (
    df_base["ciudad_residencia"]
      .astype("string")
      .str.strip()
      .str.lower()
)

# 3) Merge por ciudad
df_enriched = df_base.merge(
    kpis_city,
    left_on="ciudad_residencia",
    right_on=kpis_city.index,
    how="left"
)

df_enriched.head(5)


,yelp_n_businesses,yelp_avg_rating,yelp_top_category
city,,,
coral gables,5,4.52,Italian
doral,1,4.20,Seafood
miami,44,4.37,Cuban


,id_persona,nombre,apellido,edad,genero,ciudad_residencia,estrato_socioeconomico,frecuencia_visita,promedio_gasto_comida,ocio,consume_licor,preferencias_alimenticias,membresia_premium,telefono_contacto,correo_electronico,tipo_de_pago_mas_usado,ingresos_mensuales,yelp_n_businesses,yelp_avg_rating,yelp_top_category
0,2550327378,Jackson,Gomez,31.0,Masculino,miami,Alto,6,67.51,sí,no,Vegetariano,sí,(830)220-1926,NaN,Efectivo,6425,44.0,4.37,Cuban
1,9446112038,Samantha,Soto,40.0,Femenino,denver,Medio,2,44.92,sí,sí,Mariscos,no,881-476-1426,NaN,Efectivo,2374,NaN,NaN,<NA>
2,3098363243,Terry,Adams,62.0,Femenino,denver,Bajo,2,9.24,sí,sí,Vegetariano,no,NaN,diana74@example.net,Efectivo,1110,NaN,NaN,<NA>
3,4013002847,James,Shannon,41.0,Masculino,boston,Alto,5,30.74,sí,sí,Carnes,sí,NaN,scottfrey@example.com,Tarjeta,6931,NaN,NaN,<NA>
4,7372911048,Susan,Jones,49.0,Femenino,san diego,Bajo,0,0.00,no,no,Carnes,no,243.248.8919,glassgary@example.org,Tarjeta,1350,NaN,NaN,<NA>


In [28]:
# === Limpieza simple de "price" y top categorías (para documentación) ===
df_yelp["price_num"] = (
    df_yelp["price"]
      .map({"$":1, "$$":2, "$$$":3, "$$$$":4})
      .astype("Int64")
)

tabla_yelp_resumen = pd.DataFrame({
    "n_businesses": [len(df_yelp)],
    "avg_rating": [df_yelp["rating"].mean().round(2)],
    "med_review_count": [df_yelp["review_count"].median()],
    "price_coverage_%": [df_yelp["price"].notna().mean() * 100]
})

top_categorias = (
    df_yelp["category_primary"]
      .value_counts(dropna=True)
      .head(10)
      .rename_axis("category_primary")
      .reset_index(name="count")
)

display(tabla_yelp_resumen)
display(top_categorias)


,n_businesses,avg_rating,med_review_count,price_coverage_%
0,50,4.38,529.0,78.0


,category_primary,count
0,Mediterranean,5
1,Cuban,4
2,Italian,3
3,Asian Fusion,3
4,Seafood,3
5,New American,3
6,Mexican,2
7,Argentine,2
8,Steakhouses,2
9,Breakfast & Brunch,1


In [29]:
# Asegura que el índice de KPIs (ciudad) esté normalizado igual que en df
kpis_city = kpis_city.copy()
kpis_city.index = (
    kpis_city.index.astype("string").str.strip().str.lower()
)

# Normaliza también en df por si acaso (ya lo tenías)
df["ciudad_residencia"] = (
    df["ciudad_residencia"].astype("string").str.strip().str.lower()
)

# Merge por ciudad usando el ÍNDICE del lado derecho
df_enriched = df.merge(
    kpis_city,
    left_on="ciudad_residencia",
    right_index=True,   # <-- clave
    how="left"
)

print(df_enriched.shape)
df_enriched.loc[
    df_enriched["ciudad_residencia"] == "miami",
    ["ciudad_residencia","yelp_n_businesses","yelp_avg_rating","yelp_top_category"]
].head()




(30000, 20)


,ciudad_residencia,yelp_n_businesses,yelp_avg_rating,yelp_top_category
0,miami,44.0,4.37,Cuban
16,miami,44.0,4.37,Cuban
37,miami,44.0,4.37,Cuban
45,miami,44.0,4.37,Cuban
47,miami,44.0,4.37,Cuban


In [30]:
# === Exportar DF enriquecido ===
out_enriched = PROJ_DIR / "data" / "processed" / "base_clientes_clean_enriched.csv"
df_enriched.to_csv(out_enriched, index=False, encoding="utf-8")
print("DF enriquecido guardado en:", out_enriched.resolve())


DF enriquecido guardado en: C:\Users\Diego Garcia\Documents\Cristian\DS-SoyHenry\Modulo 1\Código\data\processed\base_clientes_clean_enriched.csv


In [31]:
df_enriched.head()

,id_persona,nombre,apellido,edad,genero,ciudad_residencia,estrato_socioeconomico,frecuencia_visita,promedio_gasto_comida,ocio,consume_licor,preferencias_alimenticias,membresia_premium,telefono_contacto,correo_electronico,tipo_de_pago_mas_usado,ingresos_mensuales,yelp_n_businesses,yelp_avg_rating,yelp_top_category
0,2550327378,Jackson,Gomez,31.0,Masculino,miami,Alto,6,67.51,sí,no,Vegetariano,sí,(830)220-1926,NaN,Efectivo,6425,44.0,4.37,Cuban
1,9446112038,Samantha,Soto,40.0,Femenino,denver,Medio,2,44.92,sí,sí,Mariscos,no,881-476-1426,NaN,Efectivo,2374,NaN,NaN,<NA>
2,3098363243,Terry,Adams,62.0,Femenino,denver,Bajo,2,9.24,sí,sí,Vegetariano,no,NaN,diana74@example.net,Efectivo,1110,NaN,NaN,<NA>
3,4013002847,James,Shannon,41.0,Masculino,boston,Alto,5,30.74,sí,sí,Carnes,sí,NaN,scottfrey@example.com,Tarjeta,6931,NaN,NaN,<NA>
4,7372911048,Susan,Jones,49.0,Femenino,san diego,Bajo,0,0.00,no,no,Carnes,no,243.248.8919,glassgary@example.org,Tarjeta,1350,NaN,NaN,<NA>


## Análisis e interpretación del enriquecimiento Yelp (Miami)

La integración con la API de **Yelp Fusion** permitió enriquecer la base de clientes con indicadores de contexto urbano basados en la oferta gastronómica local.  
En esta etapa, se trabajó exclusivamente con la ciudad de **Miami**, la cual fue seleccionada en el *Avance 1* por su alta representatividad dentro del dataset de clientes.

---

### 1. Resultados principales de la extracción Yelp

| Variable | Significado | Interpretación práctica |
|-----------|--------------|--------------------------|
| **`yelp_n_businesses`** | Cantidad de negocios registrados en Yelp para la ciudad y término “restaurants”. | En Miami se obtuvieron **45 negocios** (de un máximo de 50 solicitados). Representa una muestra significativa de la oferta gastronómica disponible. |
| **`yelp_avg_rating`** | Calificación promedio de los negocios. | El valor promedio de **4.39** indica que la oferta local está **muy bien valorada** por los usuarios. |
| **`yelp_top_category`** | Categoría más frecuente entre los negocios. | La categoría predominante fue **“Cuban”**, lo cual refleja la **identidad cultural** de la ciudad. |
| **`price_coverage_%`** | Porcentaje de registros con dato de precios. | Con un **78% de cobertura**, se dispone de información suficiente para evaluar niveles de precio. |

---

### 2. Integración con la base de clientes

El *merge* entre la base de clientes (`base_clientes_clean.csv`) y los KPI obtenidos de Yelp (`df_yelp`) permitió asociar a cada cliente de Miami indicadores agregados de su entorno comercial.

- Cada persona con `ciudad_residencia = "miami"` ahora tiene valores en las columnas:
  - `yelp_n_businesses` → número de restaurantes detectados
  - `yelp_avg_rating` → valoración promedio de Yelp
  - `yelp_top_category` → tipo de cocina predominante
- Las demás ciudades permanecen con valores `NaN`, lo cual es correcto ya que **solo Miami** fue considerada en esta etapa.

Este proceso demuestra la **integración efectiva de fuentes externas (API REST)** con datos internos de clientes, un paso clave en la construcción de análisis multifuente.

---

### 3. Análisis exploratorio y posibles interpretaciones

Aunque los datos se limitan a una ciudad, el enriquecimiento permite obtener conclusiones preliminares:

#### a) Relación entre gasto promedio y oferta Yelp
- Si se compara el **promedio de gasto mensual** de los clientes de Miami con los de otras ciudades, puede explorarse si una **mayor valoración Yelp (4.39)** se asocia a un **mayor gasto**.
- Una relación positiva sugeriría que **la percepción de calidad del entorno gastronómico** impulsa el consumo.

#### b) Cohesión cultural y categoría predominante
- La predominancia de la categoría **Cuban** no es casual: refleja la **influencia cultural** del entorno y ofrece oportunidades de **segmentación comercial por afinidad cultural**.
- Este tipo de indicador podría utilizarse para campañas de fidelización, recomendaciones locales o segmentación de marketing basada en preferencias.

#### c) Completitud y representatividad
- La presencia de **78% de cobertura de precios** y **ratings altos** evidencia que los datos de Yelp son una **fuente confiable y actualizada** para complementar información socioeconómica o de consumo.

---

### 4. Conclusiones

- La integración de Yelp con la base de clientes permitió **enriquecer la información del dataset original** con variables contextuales relevantes.
- Miami se caracteriza por una **alta satisfacción gastronómica (4.39)** y una **fuerte identidad cultural (“Cuban”)**.
- Este avance cumple el objetivo de **conectar una API externa, procesar su salida JSON, normalizarla en pandas** e **integrarla con datos locales**.
- En avances posteriores se podrá extender este enfoque a otras ciudades y evaluar:
  - Variaciones en la oferta gastronómica.
  - Correlaciones entre **entorno comercial y comportamiento del cliente**.
  - Construcción de perfiles urbanos para estrategias de negocio basadas en datos.

---

**Resultado final:**  
El dataset enriquecido se encuentra disponible en  
`data/processed/base_clientes_clean_enriched.csv`,  
sirviendo como punto de partida para el siguiente avance analítico del proyecto.
- **Notebook de entrega:** `Avance_2_API_Yelp.ipynb`
